In [1]:
from keplergl import KeplerGl
import pandas as pd

In [2]:
'''
Define paths and parameters
Tim test
Nina test
'''
DATA_PATH = './17_18.csv'

In [3]:
# load dataset
DATASET = pd.read_csv(DATA_PATH, sep=';')

# replace Datum Uhrzeit with proper timestamp as index
DATASET['timestamp'] = DATASET.Datum.map(str) + ' ' + DATASET.Uhrzeit
DATASET = DATASET.drop(['Uhrzeit', 'Datum'], axis=1)
DATASET.set_index('timestamp', inplace=True)
print(DATASET.head(5))

                 Kommerzieller Nettoexport[MWh]  Niederlande (Export)[MWh]  \
timestamp                                                                    
01.01.2017 00:00                          9.516                      1.521   
01.01.2017 01:00                          9.569                      1.365   
01.01.2017 02:00                          9.283                      1.329   
01.01.2017 03:00                          9.379                      1.203   
01.01.2017 04:00                          8.897                      1.191   

                 Niederlande (Import)[MWh]  Schweiz (Export)[MWh]  \
timestamp                                                           
01.01.2017 00:00                        -2                    1.2   
01.01.2017 01:00                        -2                    1.2   
01.01.2017 02:00                        -2                    1.2   
01.01.2017 03:00                        -2                    1.2   
01.01.2017 04:00                       

In [ ]:
'''
Define data operation functions
'''

def shrinkTimeGran(dataset, timeInterval):
    return None

In [4]:
'''
- Define Geo Data Location for data visualization
- Welches Format brauchen die Geo-Daten?
- Wo bekommen wir die Daten her?
- Wie macht man die Balken und Blasen mit kepler?

https://www.latlong.net
'''

map_1 = KeplerGl(height=500)

import_export_data = pd.read_csv("./test_data.csv", sep=";")
map_1.add_data(data=import_export_data, name='Strom Import/Export')

map_1

User Guide: https://github.com/keplergl/kepler.gl/blob/master/docs/keplergl-jupyter/user-guide.md


KeplerGl(data={'Strom Import/Export': {'index': [0, 1, 2, 3], 'columns': ['timestamp', 'lat', 'long', 'weight'…